In [1]:
import numpy as np
import pandas as pd
from econml.grf import CausalForest, CausalIVForest, RegressionForest
from econml.dml import CausalForestDML
import scipy.special
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

c:\Users\Krishna\anaconda3\envs\gen_rfm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url_1 = r"https://www.dropbox.com/s/up0k1i6btb8qmb7/usa_00004.csv?dl=1"
#url = r"C:\Users\Krishna\Dropbox\usa_00004.csv"
data_2019 = pd.read_csv(url_1)
data_2019.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,MARST,RACE,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP
0,2019,201901,1,2019010000088,11.0,2019000000011,0,1,220001,4,...,6,2,200,4,40,3,30,9000,NaN,NaN
1,2019,201901,2,2019010000096,70.0,2019000000021,0,1,100001,3,...,6,1,100,4,40,3,30,150,NaN,NaN
2,2019,201901,3,2019010000153,20.0,2019000000031,15,4,110001,4,...,6,2,200,7,71,1,10,1400,NaN,NaN
3,2019,201901,4,2019010000198,79.0,2019000000041,15,4,110001,3,...,5,1,100,3,30,3,30,22700,NaN,NaN
4,2019,201901,5,2019010000205,53.0,2019000000051,97,4,270101,3,...,3,2,200,3,30,3,30,0,NaN,NaN


In [3]:
url_2 = r"https://www.dropbox.com/s/yep1ue5e4u2qhc3/usa_00005.csv?dl=1"
#url = r"C:\Users\Krishna\Dropbox\usa_00005.csv"
data_2015 = pd.read_csv(url_2)
data_2015.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,MARST,RACE,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP
0,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,1,2,200,5,50,1,10,3450,2.0,61.0
1,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,1,2,200,7,71,3,30,14000,1.0,56.0
2,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,6,2,200,1,15,0,0,9999999,NaN,NaN
3,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,4,2,200,7,71,1,10,35000,NaN,NaN
4,2015,201501,2,160,65.0,2015000000021,0,1,100001,1,...,4,8,802,2,26,3,30,8800,NaN,NaN


In [4]:
data_2015.columns


Index(['YEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'COUNTYFIP',
       'METRO', 'STRATA', 'GQ', 'SSMC', 'PERNUM', 'PERWT', 'SEX', 'AGE',
       'MARST', 'RACE', 'RACED', 'EDUC', 'EDUCD', 'EMPSTAT', 'EMPSTATD',
       'INCTOT', 'SEX_SP', 'AGE_SP'],
      dtype='object')

In [7]:
data_2015["SSMC"].unique()

array([0, 2, 1], dtype=int64)

In [13]:
data_2015.loc[:, 'same_sex_couple'] = 0
data_2015.loc[data_2015['SEX'] == data_2015['SEX_SP'], 'same_sex_couple'] = 1

In [15]:
data_2015['same_sex_couple'].unique()

array([0, 1], dtype=int64)

In [16]:
check_data = data_2015[['SEX',
                        'SEX_SP',
                        'same_sex_couple']].copy()
check_data

,SEX,SEX_SP,same_sex_couple
0,1,2.0,0
1,2,1.0,0
2,2,NaN,0
3,1,NaN,0
4,2,NaN,0
...,...,...,...
3147000,1,NaN,0
3147001,1,NaN,0
3147002,1,NaN,0
3147003,2,NaN,0


In [18]:
data_2015.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP,same_sex_couple,new
0,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,200,5,50,1,10,3450,2.0,61.0,0,NaN
1,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,200,7,71,3,30,14000,1.0,56.0,0,NaN
2,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,200,1,15,0,0,9999999,NaN,NaN,0,NaN
3,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,200,7,71,1,10,35000,NaN,NaN,0,NaN
4,2015,201501,2,160,65.0,2015000000021,0,1,100001,1,...,802,2,26,3,30,8800,NaN,NaN,0,NaN


In [19]:
data_2015['same_sex_couple'].value_counts()

0    3126443
1      20562
Name: same_sex_couple, dtype: int64

In [20]:
data_2019.loc[:, 'same_sex_couple'] = 0
data_2019.loc[data_2019['SEX'] == data_2019['SEX_SP'], 'same_sex_couple'] = 1
data_2019['same_sex_couple'].unique()

array([0, 1], dtype=int64)

In [21]:
check_data1 = data_2019[['SEX',
                        'SEX_SP',
                        'same_sex_couple']].copy()
check_data1

,SEX,SEX_SP,same_sex_couple
0,1,NaN,0
1,2,NaN,0
2,1,NaN,0
3,1,NaN,0
4,1,NaN,0
...,...,...,...
3239548,2,NaN,0
3239549,1,NaN,0
3239550,2,NaN,0
3239551,2,1.0,0


In [22]:
data_2019['same_sex_couple'].value_counts()

0    3216593
1      22960
Name: same_sex_couple, dtype: int64